In [1]:
import findspark 
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("ETL using Spark").getOrCreate()

In [3]:
data = [("student1",64,90),
        ("student2",59,100),
        ("student3",69,95),
        ("",70,110),
        ("student5",60,80),
        ("student3",69,95),
        ("student6",62,85),
        ("student7",65,80),
        ("student7",65,80)]

In [4]:
df = spark.createDataFrame(data, ['student', 'height_inches', 'weight_pounds'])
df.show()

+--------+-------------+-------------+
| student|height_inches|weight_pounds|
+--------+-------------+-------------+
|student1|           64|           90|
|student2|           59|          100|
|student3|           69|           95|
|        |           70|          110|
|student5|           60|           80|
|student3|           69|           95|
|student6|           62|           85|
|student7|           65|           80|
|student7|           65|           80|
+--------+-------------+-------------+



In [5]:
df.write.csv(path="D:/Agiat_Ikazinat/machine_learning/ml_apache_spark/student_hw.csv", header=True)

Py4JJavaError: An error occurred while calling o44.csv.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:735)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:270)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:286)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:660)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:700)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:699)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:6

In [ ]:
df1 = spark.read.csv("student-hw.csv", header=True, inferSchema=True)

In [ ]:
df = df.dropDuplicates()
df.show()


In [ ]:
df.count()

In [ ]:
df = df.dropna()

In [ ]:
df.show()

In [ ]:
df.write.mode("overwrite").parquet("student_hw.parquet")

In [ ]:
# condesnse parquet to a single file 
df = df.repartition(1) # reduce the number of partitions 

In [ ]:
df.write.mode("overwrite").parquet("student-hw-single.parquet")

In [ ]:
df = spark.read.parquet("student-hw-single.parquet")
df.show()

In [ ]:
from pyspark.sql.functions import expr

In [ ]:
# convert inches to centimeters
df = df.withColumn("height_centimeters", expr("height_inches  * 2.54"))
df.show()

In [ ]:
# convert punds to kilograms
df = df.withColumn("weight_kg", expr("weight_pounds * 0.453592"))
df.show()

In [ ]:
df = df.drop("height_inches", 'height_pounds')
df.show()

In [ ]:
df = df.withColumnRenamed('height_centimeters', 'height_cm')
df.show()

In [ ]:
df.write.mode("overwrite").csv("student_transformed.csv", header = True)

In [ ]:
# verify the csv file 
df = spark.read.csv("student_transformed.csv", header = True, inferSchema = True)
df.show()

In [ ]:
spark.stop()

# Exercise

In [6]:
spark1 = SparkSession.builder.appName("Exercise - ETL using Spark").getOrCreate()